In [4]:
%pip install -U langchain_zhipu==4.1.8

  Using cached langchain_zhipu-4.1.8-py3-none-any.whl.metadata (5.3 kB)
Using cached langchain_zhipu-4.1.8-py3-none-any.whl (18 kB)
  Attempting uninstall: langchain_zhipu
    Found existing installation: langchain_zhipu 4.1.6
    Uninstalling langchain_zhipu-4.1.6:
      Successfully uninstalled langchain_zhipu-4.1.6

[notice] A new release of pip is available: 24.1 -> 24.1.2
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
!pip install -U langchain-openai

# 关键概念

- 🌹 大模型
- 🌹 提示语
- 🌹 链
- 🌹 文本向量
- 🌹 检索
- 🌹 工具
- 🌹 智能体

## 大模型

In [1]:
from langchain_zhipu import ChatZhipuAI
chat = ChatZhipuAI()

In [10]:
chat.invoke("给我一个中文名字，三个字")

AIMessage(content='林子轩', response_metadata={'id': '202407161356149c4e8b4478bd45c893c5b838c0ab879a', 'created': 1721109374, 'token_usage': {'completion_tokens': 6, 'prompt_tokens': 13, 'total_tokens': 19}, 'model_name': 'glm-4-air', 'finish_reason': 'stop'}, id='run-c81e7e3a-0579-4164-a8cf-706e1b12fef3-0')

In [ ]:
for x in chat.stream("给我一个中文名字，三个字"):
    print(x.content, end="|")

In [ ]:
from langchain_community.llms import Tongyi
llm = Tongyi(model="qwen2-1.5b-instruct")
llm.invoke("给我一个中文名字，三个字")

## 提示语

In [ ]:
"abc"
who = "Adi"
f'hello {who} <f-string>'

In [ ]:
from langchain_core.prompts import PromptTemplate
prompt_template = PromptTemplate.from_template("讲一个关于 {topic} 的笑话")
prompt_template.invoke("abc")
# prompt_template.input_schema.schema()
# prompt_template.invoke({"topic": "cats"})

In [ ]:
prompt_template2 = PromptTemplate.from_template("告诉我 {topic} 的原理")

## Chain 链

In [ ]:
chain = prompt_template | llm
chain.invoke({"topic": "汽车"})

In [ ]:
chain = prompt_template2 | chat
for x in chain.stream({"topic": "汽车"}):
    print(x.content, end="")

## 文本向量

In [ ]:
from langchain_zhipu import ZhipuAIEmbeddings
embeddings_model = ZhipuAIEmbeddings()

In [ ]:
embeddings = embeddings_model.embed_documents(
    [
        "Hi there!",
        "Oh, hello!",
        "What's your name?",
        "My friends call me World",
        "Hello World!"
    ]
)
len(embeddings), len(embeddings[0])

In [ ]:
# [-0.06281121,
#  0.05147552,
#  -0.016494857,
#  0.0023994024,
#  0.02036759,
#  0.009170211,
# ...]
embeddings[1]

In [ ]:
embedded_query = embeddings_model.embed_query(
    "What was the name mentioned in the conversation?")
embedded_query[:5]

## RAG 检索

In [ ]:
from langchain_community.document_loaders import TextLoader
loader = TextLoader("txt/state_of_the_union.txt")
documents = loader.load()

from langchain_text_splitters import CharacterTextSplitter
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

from langchain_community.vectorstores import FAISS
embeddings = ZhipuAIEmbeddings()
vectorstore = FAISS.from_documents(texts, embeddings)
retriever = vectorstore.as_retriever()

In [ ]:
docs = retriever.invoke("请告诉我关于乌克兰的核心观点")
docs

In [ ]:
from langchain_core.prompts import PromptTemplate
prompt_template = PromptTemplate.from_template("""
总统这样说：
----------------
{info}
----------------
请根据上述信息用中文回答我的问题：{task}
""")
p = prompt_template.invoke({
    "info": "We see the unity among leaders of nations and a more unified Europe a more unified West. And we see unity among the people who are gathering in cities in large crowds around the world even in Russia to demonstrate their support for Ukraine.  \n\nIn the battle between democracy and autocracy, democracies are rising to the moment, and the world is clearly choosing the side of peace and security. \n\nThis is a real test. It’s going to take time. So let us continue to draw inspiration from the iron will of the Ukrainian people. \n\nTo our fellow Ukrainian Americans who forge a deep bond that connects our two nations we stand with you.",
    "task": "请告诉我关于乌克兰的核心观点"
})
print(p.text)

In [ ]:
def flat_docs(docs):
    return "\n\n".join([d.page_content for d in docs])

chain = {
    "info": retriever|flat_docs,
    "task": lambda y: y
} | prompt_template | llm

In [ ]:
for x in chain.stream("请告诉我关于乌克兰的核心观点"):
    print(x, end="")

In [ ]:
chat.invoke("请问3219的平方是多少")

## 工具

In [2]:
from langchain_core.tools import tool
from langchain_core.utils.function_calling import convert_to_openai_tool

@tool
def add(a: int, b: int) -> int:
    """Adds a and b."""
    return a + b

@tool
def multiply(a: int, b: int) -> int:
    """Multiplies a and b."""
    return a * b

tools = [add, multiply]

In [3]:
llm_with_tools = chat.bind_tools(tools)

query = "What is 3 * 12?"
llm_with_tools.invoke(query)

AIMessage(content='', additional_kwargs={'tool_calls': [{'function': {'arguments': '{"a": 3, "b": 12}', 'name': 'multiply'}, 'id': 'call_2024071613582400217c2626ec4d0cb9f3b0af77f30381', 'index': 0, 'type': 'function'}]}, response_metadata={'id': '2024071613582400217c2626ec4d0cb9f3b0af77f30381', 'created': 1721109505, 'token_usage': {'completion_tokens': 15, 'prompt_tokens': 270, 'total_tokens': 285}, 'model_name': 'glm-4-air', 'finish_reason': 'tool_calls'}, id='run-c4a08f29-6dfb-4eef-97f0-daeb19e955ea-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 12}, 'id': 'call_2024071613582400217c2626ec4d0cb9f3b0af77f30381'}])

## Agent 智能体
[https://github.com/arcstep/langchain_zhipuai/blob/main/notes/agent.ipynb]